# Training a neural network on QM9

This tutorial will explain how to use SchNetPack for training a model
on the QM9 dataset and how the trained model can be used for further.

First, we import the necessary modules and create a new directory for the data and our model.

In [1]:
import os
import schnetpack as spk
from schnetpack.datasets import QM9
import schnetpack.transform as trn

import torch
import torchmetrics
import pytorch_lightning as pl

qm9tut = './qm9tut'
if not os.path.exists('qm9tut'):
    os.makedirs(qm9tut)

## Loading the data

As explained in the [previous tutorial](tutorial_01_preparing_data.ipynb), datasets in SchNetPack are loaded with the `AtomsLoader` class or one of the sub-classes that are specialized for common benchmark datasets. 
The `QM9` dataset class will download and convert the data. We will only use the inner energy at 0K `U0`, so all other properties do not need to be loaded:

In [2]:
%rm split.npz

qm9data = QM9(
    './qm9.db', 
    batch_size=100,
    num_train=1000,
    num_val=1000,
    transforms=[
        trn.TorchNeighborList(cutoff=5.),
        trn.RemoveOffsets(QM9.U0, remove_mean=True, remove_atomrefs=True),
        trn.CastTo32()
    ],
    property_units={QM9.U0: 'eV'},
    num_workers=1,
    split_file=os.path.join(qm9tut, "split.npz")
)
qm9data.prepare_data()
qm9data.setup()

rm: cannot remove 'split.npz': No such file or directory


100%|██████████| 10/10 [00:21<00:00,  2.15s/it]


The dataset is downloaded and partitioned automatically. PyTorch `DataLoader`s can be obtained using `qm9data.train_dataloader()`, `qm9data.val_dataloader()` and `qm9data.test_dataloader()`.

Before building the model, we remove offsets from the energy for good initial conditions. We will get this from the training dataset. Above this is done automatically by the `RemoveOffsets` transform.
In the following we show what happens under the hood.
For QM9, we also have single-atom reference values stored in the metadata:

In [3]:
atomrefs = qm9data.train_dataset.atomrefs
print('U0 of hyrogen:', atomrefs[QM9.U0][1].item(), 'eV')
print('U0 of carbon:', atomrefs[QM9.U0][6].item(), 'eV')
print('U0 of oxygen:', atomrefs[QM9.U0][8].item(), 'eV')

U0 of hyrogen: -13.613121032714844 eV
U0 of carbon: -1029.863037109375 eV
U0 of oxygen: -2042.611083984375 eV


These can be used together with the mean and standard deviation of the energy per atom to initialize the model with a good guess of the energy of a molecule. When calculating these statistics, we pass the atomref to take into account, that the model will add these atomrefs to the predicted energy later, so that this part of the energy does not have to be considered in the statistics, i.e.
\begin{equation}
\mu_{U_0} = \frac{1}{n_\text{train}} \sum_{n=1}^{n_\text{train}} \left( U_{0,n} - \sum_{i=1}^{n_{\text{atoms},n}} U_{0,Z_{n,i}} \right)
\end{equation}
for the mean and analog for the standard deviation. In this case, this corresponds to the mean and std. dev of the *atomization energy* per atom.

In [4]:
means, stddevs = qm9data.get_stats(
    QM9.U0, divide_by_atoms=True, remove_atomref=True
)
print('Mean atomization energy / atom:', means.item())
print('Std. dev. atomization energy / atom:', stddevs.item())

Mean atomization energy / atom: -4.248723880126298
Std. dev. atomization energy / atom: 0.19338399600016018


## Setting up the model

Next, we need to build the model and define how it should be trained

In SchNetPack, a model usually consists of two parts:
1. The representation which either constructs atom-wise features, e.g. with SchNet or PaiNN.
2. One or more output modules for property prediction.

Here, we use the `SchNet` representation with 3 interaction layers, a 5 Angstrom cosine cutoff with pairwise distances
expanded on 20 Gaussians and 50 atomwise features and convolution filters here, since we only have a few
training examples. Then, we use an `Atomwise` module to predict the inner energy $U_0$ by summing over atom-wise
energy contributions.

In [5]:
cutoff = 5.
n_atom_basis = 30

radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
schnet = spk.representation.SchNet(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)

pred_U0 = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=QM9.U0)
output_U0 = spk.atomistic.ModelOutput(
    name=QM9.U0,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=1.,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

The output model stores the prediction in the result dictionary under the `output_key`, which is mapped to a
a target with loss function using the `ModelOutput`.
By default, the target is assumed to have the same name as the output. Otherwise, a different `target_name`
has to be provided.
Here, we already gave the output the same name as the target in the dataset (`QM9.U0`).
In case of multiple outputs, the full loss is a weighted sum of all output losses.
Therefore, it is possible to provide a `loss_weight`, which we here just set to 1.

All components defined above are then passed to `AtomisticModel`, which is a sublass of
[`LightningModule`](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html).
This defines the model and training process and can then be passed to the PyTorch Lightning `Trainer`.

In [6]:
model = spk.atomistic.AtomisticModel(
    representation=schnet,
    output_modules=[pred_U0],
    outputs=[output_U0],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr":5e-4},
    postprocess=[trn.CastTo64(), trn.AddOffsets(QM9.U0, add_mean=True, add_atomrefs=True)]
)

The last part here is a list of postprocessors that will only be used if `model.inference_mode=True` is set.
This will not be used in training or validation, but only for predictions.
Here, this is used to deal with numerical accuracy and normalization of model outputs:
To make training easier, we have substracted single atom energies as well as the mean energy per atom
in the preprocessing (see above).
This does not matter for the loss, but for the final prediction we want to get the real energies.
Additionally, we have removed the energy offsets *before* casting to float32 in the preprocessor.
This avoids loss of numerical precision.
Analog to this, we also have to first cast to float64, before re-adding the offsets in the post-processor

## Training the model

Now, the model is ready for training. Since we already defined all necessary components, wthe only thing left to do is
passing it to the PyTorch Lightning `Trainer` together with the data module.

Additionally, we can provide callback that take care of logging, checkpointing etc.

In [7]:
logger = pl.loggers.TensorBoardLogger(save_dir=qm9tut)
callbacks = [
    spk.train.ModelCheckpoint(
        inference_path=os.path.join(qm9tut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=qm9tut,
    max_epochs=3, # for testing, we restrict the number of epochs
)
trainer.fit(model, datamodule=qm9data)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/kschuett/anaconda3/envs/spkdev/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1303: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
/home/kschuett/anaconda3/envs/spkdev/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name           | Type       | Params
----------------------------------------------
0 | representation | SchNet     | 16.0 K
1 | outputs        | ModuleList | 0     
2 | output_modules | ModuleList | 481   
3 | postprocessors | ModuleList | 0     
----------------------------------------------
16.4 K    Trainable params
0         Non-traina

Validation sanity check: 0it [00:00, ?it/s]

/home/kschuett/anaconda3/envs/spkdev/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kschuett/anaconda3/envs/spkdev/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kschuett/anaconda3/envs/spkdev/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (10) is smaller than the logging interval Trainer(log_ever

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

The `ModelCheckpoint` of SchNetPack is equivalent to that in PyTorch Lightning,
except that we also store the best inference model. We will show how to use thi in the next section.

You can have a look at the log using Tensorboard:
```
tensorboard --logdir=qm9tut/default
```



## Inference

Having trained a model for QM9, we are going to use it to obtain some predictions.
First, we need to load the model. The `Trainer` stores the best model in the model directory which can be loaded using PyTorch:

In [8]:
import torch
import numpy as np
from ase import Atoms

best_model = torch.load(os.path.join(qm9tut, 'best_inference_model'))

We can use the test dataloader from the QM( data to obtain a batch of molecules and apply the model:

In [9]:
for batch in qm9data.test_dataloader():
    result = best_model(batch)
    print("Result dictionary:", result)
    break

Result dictionary: {'energy_U0': tensor([ 9333.5307, 11332.8081, 10844.4088, 11210.0076, 11836.1201,  9172.6574,
        10287.7766, 11334.1123, 11332.9134, 10824.0110, 11785.6308, 11212.4274,
        10235.3932, 12243.2745, 12345.7868, 12345.3807, 11796.5354, 12224.1717,
        11371.3230, 10751.6529, 10649.7008, 11249.6271, 11798.1534, 12791.7377,
        11779.5039, 10358.9509, 10772.0780, 11351.8941, 11231.2139, 10783.5423,
         9799.1497, 12242.0594, 11231.4712, 11321.1248, 11372.1191, 11818.0016,
        11675.4923, 10203.5050, 10379.0468, 11230.4986,  8132.8889, 11779.0077,
        11637.8801, 11372.0278,  9778.6330, 11211.0127, 11230.3103, 12243.3340,
        10359.2251,  9771.6275, 12405.5860, 10785.0830, 11372.3786, 12345.5302,
        11321.4495, 12790.7953, 11248.1417, 12261.9512, 12389.4074,  9274.6194,
        12365.6048, 10358.9336,  9333.0098, 12706.1660, 11778.6273, 11740.6169,
        11216.4509, 10378.7661, 10784.3455,  9346.2143, 10784.9676, 11371.9886,
       

If your data is not already in SchNetPack format, a convenient way is to use ASE atoms with the
provided `AtomsConverter` and (optionally) the `SpkCalculator`:

In [10]:
converter = spk.interfaces.AtomsConverter(neighbor_list=trn.ASENeighborList(cutoff=5.), dtype=torch.float32)
calculator = spk.interfaces.SpkCalculator(model=best_model, converter=converter)

In [11]:
numbers = np.array([6, 1, 1, 1, 1])
positions = np.array([[-0.0126981359, 1.0858041578, 0.0080009958],
                      [0.002150416, -0.0060313176, 0.0019761204],
                      [1.0117308433, 1.4637511618, 0.0002765748],
                      [-0.540815069, 1.4475266138, -0.8766437152],
                      [-0.5238136345, 1.4379326443, 0.9063972942]])
atoms = Atoms(numbers=numbers, positions=positions)

In [12]:
inputs = converter(atoms)

print('Keys:', list(inputs.keys()))

pred = best_model(inputs)

print('Prediction:', pred[QM9.U0])

Keys: ['_n_atoms', '_atomic_numbers', '_positions', '_cell', '_pbc', '_idx_i_local', '_idx_j_local', '_Rij', '_idx_m', '_idx_j', '_idx_i']
Prediction: tensor([1062.9546], dtype=torch.float64, grad_fn=<SubBackward0>)


Alternatively, one can use the `SpkCalculator` as an interface to ASE:

In [14]:
calculator = spk.interfaces.SpkCalculator(model=best_model, converter=converter, energy=QM9.U0, energy_units="eV")
atoms.set_calculator(calculator)
print('Prediction:', atoms.get_total_energy())

Prediction: 1062.9545620828867


The calculator automatically converts the prediction of the given unit to internal ASE units, which is `eV`
for the energy.
Using the calculator interface makes more sense if you have trained SchNet for a potential energy surface.
In the next tutorial, we will show how to learn potential energy surfaces and forces field as well as performing
molecular dynamics simulations with SchNetPack.